## Error handling is boring but this is the Exception

### Vito Minheere

# What is an Exception?

In [43]:
raise "Exception" 

TypeError: exceptions must derive from BaseException

An Exception is an object as many other things in Python. Exception extends the BaseException.

Extend it to create other Exceptions with the same attributes

Internal Python Exceptions should use BaseException, all others should use Exception



# Exception hierarchy

BaseException
 ├── GeneratorExit
 ├── KeyboardInterrupt
 ├── SystemExit
 └── Exception
      ├── ArithmeticError
      │    ├── FloatingPointError
      │    ├── OverflowError
      │    └── ZeroDivisionError
      ├── AttributeError
      ├── EOFError
      ├── LookupError
      │    ├── IndexError
      │    └── KeyError

Built-in errors also have their hierarchy. 

Exception hierarchy can serve an important purpose: 

the user does not have to know all the specific exceptions 

## Handling Exceptions: Raising an Exception

In [44]:
1 / 0

ZeroDivisionError: division by zero

## Handling Exceptions: Using except

In [45]:
try:
    1 / 0
    
except ZeroDivisionError as e:
    print("Zero Division raised")

Zero Division raised


## Handling Exceptions: Order of handling

In [46]:
try:
    1 / 0
    
except ArithmeticError as e:
    print("Arithmetic Error raised")
    raise e
    
except ZeroDivisionError as e:
    print("Zero Division raised")
    raise e

Arithmetic Error raised


ZeroDivisionError: division by zero

Arithmetic error is higher on the hierarchy than ZeroDivisionError

At most one except clause will trigger

The first except clause that matches will trigger


## Handling Exceptions: Order of handling

In [47]:
try:
    1 / 0

except Exception as e:
    print("Exception raised")
    
except ArithmeticError as e:
    print("Arithmetic Error raised")
    raise e
    
except ZeroDivisionError as e:
    print("Zero Division raised")
    raise e       


Exception raised


except Exception is even higher and will be executed first


## Handling Exceptions: Dangerous handling

In [48]:
try:
    thisssss_function_does_not_exist()
    
except Exception as e:
    pass
    
except ArithmeticError as e:
    print("Arithmetic Error raised")
    raise e
    
except ZeroDivisionError as e:
    print("Zero Division raised")
    raise e


Be aware that Exception will also catch all other Exceptions derived from it

Python is an interpreted language.
Python uses Exceptions for both reporting errors while running the code and interpreting code. 
If you catch all Exceptions it will also catch NameErrors during interpreting

## Handling Exceptions: Order is important

In [49]:
try:
    raise Exception('general exceptions not caught by specific handling')
    
except ValueError as e:
    print('we will not catch exception: Exception')



Exception: general exceptions not caught by specific handling

Hierarchy only travels up not down

## Handling Exceptions: Pythonic notation

In [50]:
try:
    1 / 0

except ArithmeticError as e:
    print("Error raised")
    
except ZeroDivisionError as e:
    print("Error raised")


Error raised


In [51]:
try:
    1 / 0

except (ArithmeticError, ZeroDivisionError) as e:
    print("Error raised")
    

Error raised


# NAASA, NASA As A Service


#### Launch a rocket via a Python module. 

### No guarantees, only Exceptions

It is rocket science so a lot of things can go wrong. 

Before the launch you could exit your program and call it off if critical errors arise 

but you can’t just reset the rocket when it is in mid flight. 

In this case we are both the creator of the module and also the user.
We will use a bit of defensive programming to make sure edge cases will be handled and the rocket will not crash. 

## The Rocket

In [52]:
from classes.fuel_tank import FuelTank
from classes.command_centre import CommandCentre
from classes.engine import Engine

class Rocket:
    def __init__(self, name):
        self.name = name
        self.altitude = 0
        self.crew = []
        self.fuel_tank = FuelTank("Hydrogen", 100)
        self.command_centre = CommandCentre()
        self.engine = Engine(units=4)
        self.startup_completed = False
        
    def __str__(self):
        return f"Rocket {self.name} has {self.engine.units} engines and is at an altitude of {self.altitude} meters"
         
rocket = Rocket("Artemis 1")
print(rocket)

Rocket Artemis 1 has 4 engines and is at an altitude of 0 meters


In [ ]:
class FuelTank:
    def __init__(self, fuel_type: str, max_volume: int) -> None:
        self.fuel_type = fuel_type
        self.volume = 0
        self.max_volume = max_volume

## Custom Exceptions

In [53]:
 class FuelTankError(Exception):
        pass

raise FuelTankError("Something is wrong with the fuel tank")

FuelTankError: Something is wrong with the fuel tank

To create a custom Exception you can create a class and extend Exception

Usually you can keep the base empty so that child Exception can set their own messages without them being overwritten by the base

## Custom Exceptions: Child Exceptions

In [54]:
class FuelTypeError(FuelTankError):
    
    def __init__(self, fuel_type, allowed_fuel):
        message = f"Invalid fuel type: {fuel_type}. Only '{allowed_fuel}' fuel is allowed."
        super().__init__(message)

class FuelLevelError(FuelTankError):  
    
    def __init__(self, max_volume, volume):
        amount_left_to_fill = max_volume - volume
        message = f"Max amount of fuel to add is {str(amount_left_to_fill)} liters" 
        self.value = amount_left_to_fill
        super().__init__(message)


raise FuelLevelError(10, 7)

FuelLevelError: Max amount of fuel to add is 3 liters

## Custom Exceptions: Adding them to code

In [55]:
def refuel(self, fuel_type, amount):
    if fuel_type != self.fuel_type:
        raise FuelTypeError(fuel_type)
        
    elif (self.volume + amount) > self.max_volume:
        raise FuelLevelError(self.max_volume, self.volume)
        
    else:
        self.volume += amount
        print(f"Fuel tank refueled. Fuel level is now {self.volume} liter.")

Build a hierarchy in the custom exceptions. 

FuelException will be caught before FuelTankException when both are in the except clause.



## Custom Exceptions: Handling them

In [60]:
from classes.fuel_tank import FuelTankError, FuelTypeError, FuelLevelError

try:
    rocket.fuel_tank.refuel("Diesel", 100)

except FuelLevelError as e:
    print("FuelLevelError")
    raise e
    
except FuelTypeError as e:
    print("FuelTypeError")
    raise e

FuelTypeError


FuelTypeError: Invalid fuel type: Diesel. Only 'Hydrogen' fuel is allowed.

Due to the hierarchy the base FuelTankException can be caught first. 

When you reraise the Exception it shows the FuelTypeException.

End users of our module can put it as the last except block to catch any errors they might have missed.

## Custom Exceptions: Order is still important

In [61]:
try:
    rocket.fuel_tank.refuel("Diesel", 100)
    
except FuelLevelError as e:
    print("FuelLevelError")
    raise e
    
except FuelTankError as e:
    print("FuelTankError")
    raise e

FuelTankError


FuelTypeError: Invalid fuel type: Diesel. Only 'Hydrogen' fuel is allowed.

Here is an example of setting the custom "base" exception in the last block

The specific exception is missed in the code handling it

The hierarchy still manages to catch it and display the correct Exception happening

## Explicit chained exceptions: Keeping track of Exceptions

In [62]:
from classes.rocket import Rocket

class RocketNotReadyError(Exception):
    pass

def refuel(rocket):
    try:
        rocket.refuel("Diesel", 100)
    except FuelTankError as e:
        raise RocketNotReadyError("Can't launch Rocket today") from e

An Exception can be caught and reraised again in the except clause

You can also use raise from to raise a new Exception but keep the old one in the stack trace

## Explicit chained exceptions: Showing the traceback

In [63]:
from classes.rocket import RocketNotReadyError

try:
    print('Final check procedure')
    rocket = Rocket("Artemis 2")
    refuel(rocket)
except RocketNotReadyError as f:
    print('General exception: "{}", caused by "{}"'.format(f, f.__cause__))
    raise f

Final check procedure
General exception: "Can't launch Rocket today", caused by "Invalid fuel type: Diesel. Only 'Hydrogen' fuel is allowed."


RocketNotReadyError: Can't launch Rocket today

The raise from can be extended again in another try except

The Exception will travel up the chain to where it wil be caught

See the traceback in the slide

## Suppressing errors

In [64]:
fuels = ["Diesel", "Hydrogen"]

fuels.remove("Gasoline")

ValueError: list.remove(x): x not in list

## Suppressing errors

In [69]:
fuels = ["Diesel", "Hydrogen"]

# (LBYL) Look before you leap:
def remove_from_list(a_list, element):
    if element in a_list:
        a_list.remove(element)
         
remove_from_list(fuels, "Gasoline")
print(fuels)

['Diesel', 'Hydrogen']


## Suppressing errors: Try Except pass

In [70]:
fuels = ["Diesel", "Hydrogen"]

# (EAFP) Easier to ask forgiveness than permission:
def remove_from_list(a_list, element):
    try:
        a_list.remove(element)
    except ValueError:
        pass

remove_from_list(fuels, "Gasoline")
print(fuels)

['Diesel', 'Hydrogen']


## Suppressing errors: Using contextlib

In [71]:
fuels = ["Diesel", "Hydrogen"]

from contextlib import suppress

def remove_from_list(a_list, element):
    with suppress(ValueError):
        a_list.remove(element)
        
remove_from_list(fuels, "Gasoline")
print(fuels)

['Diesel', 'Hydrogen']


Easier to Ask for Forgiveness than Permission

As with any other mechanism that completely suppresses exceptions, this context manager should be used only to cover very specific errors where silently continuing with program execution is known to be the right thing to do.

## Suppressing errors: Logging

In [72]:
fuels = ["Diesel", "Hydrogen"]

def remove_from_list(a_list, element):
    try:
        a_list.remove(element)
    except ValueError:
        print(f"{element} not in list")

remove_from_list(fuels, "Gasoline")
print(fuels)

Gasoline not in list
['Diesel', 'Hydrogen']


## Suppressing errors: Using the Rocket example

In [73]:
class CommandCentre:
    def __init__(self) -> None:
        self.signal = 100
        
    def generate_transmissions(self):
        messages = [
            "Message 1",
            {"number": 5},
            "Message 2",
            5,
            "Message 3",
        ]
        for message in messages:
            yield message  

The command centre will broadcast messages

Only the strings are interesting to us. The other will give Exceptions when we try to handle them

What could be a clean way to handle these?

## Suppressing errors: Try Except pass

In [74]:
def receive_transmissions():
    centre = CommandCentre()
    transmission_generator = centre.generate_transmissions()
    while True:
        try:
            transmission = next(transmission_generator)
            
            try:
                transmission = transmission.strip()
            except AttributeError:
                continue
            except TypeError:
                continue
            else:
                print(f"Received: {transmission}")
                
        except StopIteration:
            print("End of transmission")
            break
        
receive_transmissions()

Received: Message 1
Received: Message 2
Received: Message 3
End of transmission


## Suppressing errors: Using contextlib

In [75]:
from contextlib import suppress

def receive_transmissions():
    centre = CommandCentre()
    transmission_generator = centre.generate_transmissions()    
    while True:
        try:
            transmission = next(transmission_generator)
            
            with suppress((AttributeError, TypeError)):
                transmission = transmission.strip()
                print(f"Received: {transmission}")
                
        except StopIteration:
            print("End of transmission")
            break

receive_transmissions()

Received: Message 1
Received: Message 2
Received: Message 3
End of transmission


With suppress uses a ContextManager to handle the Exceptions. The result is the same as the code in the previous slide but it is easier to read and more Pythonic.

Return a context manager that suppresses any of the specified exceptions if they occur in the body of a with statement and then resumes execution with the first statement following the end of the with statement.

# Async Exception Handling

Async Python brings a whole load of other problems including multiple exceptions from different tasks

Python 3.11 brought new features including TaskGroups and ExceptionGroups

Both named Groups and released at the same time. They must be related.



In [76]:
issubclass(ExceptionGroup, Exception)

True

## Async Exception Handling: Custom Exceptions

In [105]:
class EngineError(Exception):
    pass

class IgnitionError(EngineError):
    def __init__(self, engine) -> None:
        self.engine = engine
        message = f"Engine {engine} failed to ignite"
        super().__init__(message)

    def __str__(self):
        return f"Error in engine {self.engine}"

## Async Exception Handling: Random Engine failure

In [106]:
import asyncio
import random

async def fire_engine(engine):
    print(f"Attempting to fire engine {engine}")
    engine_fires_chance = random.random()
    
    if engine_fires_chance < 0.3:
        raise EngineError(f"Something went wrong in engine {engine}")
        
    elif engine_fires_chance >= 0.8:
        print(f"Engine {engine} failed to fire")
        raise IgnitionError(engine)
        
    print(f"Engine {engine} fired up!")

## Async Exception Handling: Creating tasks

In [183]:
async def apply_throttle(engine):
    try:
        task = asyncio.create_task(fire_engine(engine))
        await task
        if task.done():
            ex = task.exception()
            if ex:
                raise ex
    
    except IgnitionError as e:
        print("Handling IgnitionError")
        
    except EngineError as e:
        print("Handling EngineError")
        
await apply_throttle(1)
await apply_throttle(2)
await apply_throttle(3)
await apply_throttle(4)

Attempting to fire engine 1
Engine 1 fired up!
Attempting to fire engine 2
Engine 2 fired up!
Attempting to fire engine 3
Handling EngineError
Attempting to fire engine 4
Engine 4 fired up!


## Async Exception Handling: Gathering Tasks

In [186]:
async def apply_throttle():
    try:
        results = await asyncio.gather(
            fire_engine(1), 
            fire_engine(2), 
            fire_engine(3), 
            fire_engine(4)
        )
    except IgnitionError as e:
        print("Handling IgnitionError")
        
    except EngineError as e:
        print("Handling EngineError")
                
await apply_throttle()

Attempting to fire engine 1
Engine 1 failed to fire
Attempting to fire engine 2
Engine 2 fired up!
Attempting to fire engine 3
Engine 3 fired up!
Attempting to fire engine 4
Engine 4 fired up!
Handling IgnitionError


When a co-routine raises an exception it propagates up to the scheduler. 
The scheduler captures the exception, marks the task as done, removes it from the event loop and notes the exception in the task object.

## Async Exception Handling: Using TaskGroups

In [187]:
async def apply_throttle(motors):
    try:
        async with asyncio.TaskGroup() as tg:
            
            for x in range(motors):
                tg.create_task(
                    fire_engine(x)
                )

    except* IgnitionError as e:
        print(f"Handling IgnitionErrors: {e.exceptions}")
        
    except* EngineError as e:
        print(f"Handling EngineErrors: {e.exceptions}")

## Async Exception Handling: Tracebacks

In [172]:

await apply_throttle(4) 

Attempting to fire engine 0
Engine 0 fired up!
Attempting to fire engine 1
Engine 1 failed to fire
Attempting to fire engine 2
Engine 2 fired up!
Attempting to fire engine 3
Engine 3 fired up!
Handling IgnitionErrors: (IgnitionError('Engine 1 failed to ignite'),)


  + Exception Group Traceback (most recent call last):
  | ExceptionGroup:  (2 sub-exceptions)
  +-+---------------- 1 ----------------
    | Exception Group Traceback (most recent call last):
    |   File "/classes/engine.py", line 32, in apply_throttle
    |     async with asyncio.TaskGroup() as tg:
    |   File "/usr/lib/python3.11/asyncio/taskgroups.py", line 147, in __aexit__
    |     raise me from None
    | ExceptionGroup: unhandled errors in a TaskGroup (2 sub-exceptions)


   | ExceptionGroup: unhandled errors in a TaskGroup (2 sub-exceptions)
   +-+---------------- 1 ----------------
      | Traceback (most recent call last):
      |   File "/classes/engine.py", line 53, in fire_engine
      |     raise IgnitionError(motor)
      | classes.engine.IgnitionError: Error in engine 1
      +---------------- 2 ----------------
      | Traceback (most recent call last):
      |   File "/classes/engine.py", line 53, in fire_engine
      |     raise IgnitionError(motor)
      | classes.engine.IgnitionError: Error in engine 2

## So can we finally launch the Rocket?

In [ ]:
from classes.rocket import Rocket, StartUpAbort, RocketNotReadyError
from classes.launchpad import Launchpad
from classes.engine import IgnitionError

rocket = Rocket("Artemis")
launchpad = Launchpad()

try:
    rocket.startup_sequence()
except StartUpAbort as e:
    raise e


In [ ]:
async def launch_rocket(rocket):
    try:
        launchpad.clear()
        await rocket.launch()
    except ExceptionGroup as e:
        print("Launch failed")
    else:
        print("\n")
        print(rocket)
    finally:
        launchpad.release()

await launch_rocket(rocket)